<a href="https://colab.research.google.com/github/annapetrbok/ee467wi2026/blob/main/Breaking_CAPTCHAS_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os, cv2, numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from imutils import paths

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
import os, pickle, glob, math
from pprint import pprint

import cv2
import numpy as np
import imutils
from imutils import paths
from tensorflow.keras import Sequential, layers
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from lab_2_helpers import *

In [7]:
# Dataset images folder
CAPTCHA_IMAGE_FOLDER = "./captcha-images"

# List of all the captcha images we need to process
captcha_image_paths = list(paths.list_images(CAPTCHA_IMAGE_FOLDER))
# Review image paths
pprint(captcha_image_paths[:10])

!tar -xJf captcha-images.tar.xz
!ls -la
!find . -maxdepth 2 -type d | sed -n '1,120p'

['./captcha-images/J3NJ.png',
 './captcha-images/2QRH.png',
 './captcha-images/E7ZC.png',
 './captcha-images/2GHE.png',
 './captcha-images/L4M9.png',
 './captcha-images/M4YZ.png',
 './captcha-images/M4PD.png',
 './captcha-images/8CRP.png',
 './captcha-images/C4NZ.png',
 './captcha-images/M4BT.png']
total 1080
drwxr-xr-x 1 root root    4096 Jan 30 18:58 .
drwxr-xr-x 1 root root    4096 Jan 30 18:46 ..
drwxr-xr-x 2 1000 1000   32768 Dec 15  2017 captcha-images
-rw-r--r-- 1 root root 1044556 Jan 30 18:55 captcha-images.tar.xz
drwxr-xr-x 4 root root    4096 Jan 16 14:24 .config
-rw-r--r-- 1 root root    3603 Jan 30 18:51 lab_2_helpers.py
drwxr-xr-x 2 root root    4096 Jan 30 18:55 __pycache__
drwxr-xr-x 1 root root    4096 Jan 16 14:24 sample_data
.
./.config
./.config/configurations
./.config/logs
./__pycache__
./captcha-images
./sample_data


In [8]:
import os
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from lab_2_helpers import resize_to_fit  # you already have lab_2_helpers.py

# --- settings ---
TVT_SPLIT_SEED = 31528476
CAPTCHA_IMAGE_FOLDER = "./captcha-images"
CHAR_IMAGE_FOLDER = f"./char-images-{TVT_SPLIT_SEED}"
FORCE_EXTRACT_CHAR = True  # set False if you want to reuse if already exists

# --- load captcha paths + texts from filenames ---
captcha_image_paths = list(paths.list_images(CAPTCHA_IMAGE_FOLDER))

def extract_captcha_text(image_path):
    image_file_name = os.path.basename(image_path)
    return os.path.splitext(image_file_name)[0]

captcha_texts = [extract_captcha_text(p) for p in captcha_image_paths]

# --- load + grayscale + padding (same as TF) ---
def load_transform_image(image_path):
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_padded = cv2.copyMakeBorder(image_gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    return image_padded

captcha_images = [load_transform_image(p) for p in captcha_image_paths]

# --- train/val vs test split (same seed) ---
captcha_images_tv, captcha_images_test, captcha_texts_tv, captcha_texts_test = train_test_split(
    captcha_images, captcha_texts, test_size=0.2, random_state=TVT_SPLIT_SEED, shuffle=True
)

print("Train-validation:", len(captcha_texts_tv))
print("Test:", len(captcha_texts_test))

# --- segmentation functions (same as TF) ---
def extract_chars(image):
    image_bw = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(image_bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    char_regions = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            char_regions.append((x, y, half_width, h))
            char_regions.append((x + half_width, y, half_width, h))
        else:
            char_regions.append((x, y, w, h))

    if len(char_regions) != 4:
        return None

    char_regions.sort(key=lambda x: x[0])

    char_images = []
    for x, y, w, h in char_regions:
        # safe slicing (padding usually prevents issues, but clamp anyway)
        x0 = max(x - 2, 0); y0 = max(y - 2, 0)
        x1 = min(x + w + 2, image.shape[1]); y1 = min(y + h + 2, image.shape[0])
        char_images.append(image[y0:y1, x0:x1])

    return char_images

def save_chars(char_images, captcha_text, save_dir, char_counts):
    for char_image, char in zip(char_images, captcha_text):
        save_path = os.path.join(save_dir, char)
        os.makedirs(save_path, exist_ok=True)

        char_count = char_counts.get(char, 1)
        char_image_path = os.path.join(save_path, f"{char_count}.png")
        cv2.imwrite(char_image_path, char_image)
        char_counts[char] = char_count + 1

# --- run extraction ---
if FORCE_EXTRACT_CHAR or not os.path.exists(CHAR_IMAGE_FOLDER):
    # if folder exists but empty, also rebuild
    if os.path.exists(CHAR_IMAGE_FOLDER):
        # optional: clear it to avoid mixing partial runs
        import shutil
        shutil.rmtree(CHAR_IMAGE_FOLDER)

    char_counts = {}
    os.makedirs(CHAR_IMAGE_FOLDER, exist_ok=True)

    n_saved = 0
    n_skipped = 0
    for img, text in zip(captcha_images_tv, captcha_texts_tv):
        chars = extract_chars(img)
        if chars is None:
            n_skipped += 1
            continue
        save_chars(chars, text, CHAR_IMAGE_FOLDER, char_counts)
        n_saved += 1

    print("CAPTCHAs processed:", len(captcha_images_tv))
    print("Saved (segmented OK):", n_saved)
    print("Skipped (seg failed):", n_skipped)

# --- verify ---
from imutils import paths as ipaths
print("Character images now found:", len(list(ipaths.list_images(CHAR_IMAGE_FOLDER))))
print("Sample paths:", list(ipaths.list_images(CHAR_IMAGE_FOLDER))[:5])


Train-validation: 908
Test: 228
CAPTCHAs processed: 908
Saved (segmented OK): 892
Skipped (seg failed): 16
Character images now found: 3568
Sample paths: ['./char-images-31528476/X/26.png', './char-images-31528476/X/39.png', './char-images-31528476/X/22.png', './char-images-31528476/X/49.png', './char-images-31528476/X/42.png']


In [9]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from imutils import paths

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

CHAR_IMAGE_FOLDER = "./char-images-31528476"

classes = sorted([
    d for d in os.listdir(CHAR_IMAGE_FOLDER)
    if os.path.isdir(os.path.join(CHAR_IMAGE_FOLDER, d))
])
print("n_classes:", len(classes), "| sample:", classes[:10])

class_to_idx = {c:i for i,c in enumerate(classes)}
idx_to_class = {i:c for c,i in class_to_idx.items()}
n_classes = len(classes)

class CharDataset(Dataset):
    def __init__(self, paths_list, class_to_idx):
        self.paths = paths_list
        self.class_to_idx = class_to_idx

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (20, 20), interpolation=cv2.INTER_AREA)
        x = torch.tensor(img, dtype=torch.float32).unsqueeze(0) / 255.0
        y = self.class_to_idx[path.split(os.path.sep)[-2]]
        return x, y

all_paths = list(paths.list_images(CHAR_IMAGE_FOLDER))
train_paths, val_paths = train_test_split(all_paths, test_size=0.25, random_state=955996, shuffle=True)

train_loader = DataLoader(CharDataset(train_paths, class_to_idx), batch_size=32, shuffle=True)
val_loader   = DataLoader(CharDataset(val_paths, class_to_idx), batch_size=32, shuffle=False)

print("train chars:", len(train_paths), "| val chars:", len(val_paths))


device: cpu
n_classes: 32 | sample: ['2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B']
train chars: 2676 | val chars: 892


In [10]:
class CaptchaCNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 20, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(20, 50, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(50*5*5, 500),
            nn.ReLU(),
            nn.Linear(500, n_classes),
        )

    def forward(self, x):
        return self.fc(self.conv(x))

model = CaptchaCNN(n_classes).to(device)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, 11):
    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    print(f"Epoch {epoch:02d} | val_acc = {correct/total:.4f}")


Epoch 01 | val_acc = 0.9103
Epoch 02 | val_acc = 0.9798
Epoch 03 | val_acc = 0.9899
Epoch 04 | val_acc = 0.9933
Epoch 05 | val_acc = 0.9933
Epoch 06 | val_acc = 0.9933
Epoch 07 | val_acc = 0.9933
Epoch 08 | val_acc = 0.9933
Epoch 09 | val_acc = 0.9933
Epoch 10 | val_acc = 0.9933


In [12]:
import numpy as np
import torch
import cv2

# ---- helper: make a (1,20,20) tensor from a single char image ----
def make_char_tensor(char_img):
    # char_img is a 2D grayscale numpy array
    img = cv2.resize(char_img, (20, 20), interpolation=cv2.INTER_AREA)
    x = torch.tensor(img, dtype=torch.float32).unsqueeze(0) / 255.0  # (1,20,20)
    return x

# ---- build a flat list of character tensors for ALL test CAPTCHAs ----
extract_failed_indices = []
char_tensors = []

for i, captcha_img in enumerate(captcha_images_test):
    chars = extract_chars(captcha_img)  # should return list of 4 images or None
    if chars is None:
        extract_failed_indices.append(i)
        # add 4 dummy chars (won't matter; we will overwrite result with "-")
        for _ in range(4):
            char_tensors.append(torch.zeros((1, 20, 20), dtype=torch.float32))
    else:
        for ch in chars:
            char_tensors.append(make_char_tensor(ch))

# Stack into one batch: (N_chars, 1, 20, 20)
X_test_chars = torch.stack(char_tensors, dim=0)

# ---- run model prediction ----
model.eval()
with torch.no_grad():
    logits = model(X_test_chars.to(device))
    pred_idx = logits.argmax(dim=1).cpu().numpy()

# Convert predicted indices to characters
pred_chars = [idx_to_class[i] for i in pred_idx]

# Group every 4 predicted characters into a CAPTCHA string
pred_texts = ["".join(pred_chars[i:i+4]) for i in range(0, len(pred_chars), 4)]

# Overwrite CAPTCHAs where extraction failed
for i in extract_failed_indices:
    pred_texts[i] = "-"

# ---- compute CAPTCHA accuracy ----
n_test = len(captcha_texts_test)
n_correct = sum(p == t for p, t in zip(pred_texts, captcha_texts_test))

print("# of test CAPTCHAs:", n_test)
print("# correctly recognized:", n_correct)
print("Accuracy:", n_correct / n_test)
print("Extraction failed (marked '-'):", len(extract_failed_indices))


# of test CAPTCHAs: 228
# correctly recognized: 219
Accuracy: 0.9605263157894737
Extraction failed (marked '-'): 3


In [ ]:
#PyTorch results: CAPTCHA test accuracy = 0.9605 (219/228).
#TensorFlow results: CAPTCHA test accuracy ≈ 0.9386 (214/228).
#The differences are due to the randomness, initialization, and implementation details. Both frameworks achieve similar performance.